<a href="https://colab.research.google.com/github/ShamsRupak/ai-doc-processing-suite/blob/main/Document_Classification_Before_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required libraries with CUDA support
!pip install -q torch

In [2]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

CUDA available: True
GPU: Tesla T4


In [3]:
# Check CUDA version first
!nvcc --version

# Install llama-cpp-python with CUDA 12.x support
!pip install --no-cache-dir llama-cpp-python==0.2.90 --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu123

# install remaining libraries
!pip install llama-index
!pip install pymupdf
!pip install llama-index-llms-llama-cpp
!pip install llama-index-embeddings-huggingface

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu123
  Using cached llama_index_llms_llama_cpp-0.4.0-py3-none-any.whl.metadata (4.5 kB)
  Using cached llama_cpp_python-0.3.14.tar.gz (51.0 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Using cached llama_index_llms_llama_cpp-0.4.0-py3-none-any.whl (7.5 kB)
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.14-cp311-cp311-linux_x86_64.whl size=4299364 sha256=4c4028835cabff76d0797d1fcd5e644a85df695fe3df32ad759da56f865d153b
  Stored in directory: /root/.cache/pip/wheels/3f/b6/cf/7315ec7b0149210d2d4447d9c3338b36d10e56a1ecddcd35c0
Successfully 

In [4]:
from llama_cpp import Llama
import os

# Download Mistral model if not already present
model_path = "/content/mistral.gguf"
if not os.path.exists(model_path):
    !wget https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf -O {model_path}
    print(f"Model downloaded to {model_path}")

# Verify file exists and check size
if os.path.exists(model_path):
    print(f"Model file exists. Size: {os.path.getsize(model_path) / (1024 * 1024):.2f} MB")
else:
    print("Model file not found!")


--2025-07-20 17:07:42--  https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.2-GGUF/resolve/main/mistral-7b-instruct-v0.2.Q4_K_M.gguf
Resolving huggingface.co (huggingface.co)... 18.164.174.118, 18.164.174.17, 18.164.174.55, ...
Connecting to huggingface.co (huggingface.co)|18.164.174.118|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/72/62/726219e98582d16c24a66629a4dec1b0761b91c918e15dea2625b4293c134a92/3e0039fd0273fcbebb49228943b17831aadd55cbcbf56f0af00499be2040ccf9?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27mistral-7b-instruct-v0.2.Q4_K_M.gguf%3B+filename%3D%22mistral-7b-instruct-v0.2.Q4_K_M.gguf%22%3B&Expires=1753034190&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc1MzAzNDE5MH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzcyLzYyLzcyNjIxOWU5ODU4MmQxNmMyNGE2NjYyOWE0ZGVjMWIwNzYxYjkxYzkxOGUxNWRlYTI2MjViNDI5M2MxMzRhOTIvM2UwMDM5ZmQwMjczZmNiZW

In [5]:
import fitz  # PyMuPDF

# Define document paths
doc_paths = {
    "Unknown 1": "/content/sample_bank_statement.pdf",
    "Unknown 2": "/content/payslip_sample_image.pdf",
    "Unknown 3": "/content/appraisal_report.pdf"
}

# Extract text from all PDFs
doc_texts = {}

for i, (doc_type, path) in enumerate(doc_paths.items()):
    doc = fitz.open(path)
    text = "\n".join([page.get_text() for page in doc])
    doc_texts[f"Doc-{i+1}"] = text  # Temporarily label them "Unknown"
    print(f"Extracted {len(text.split())} words from {path}.")

Extracted 287 words from /content/sample_bank_statement.pdf.
Extracted 82 words from /content/payslip_sample_image.pdf.
Extracted 6470 words from /content/appraisal_report.pdf.


In [6]:
doc_texts.keys()

dict_keys(['Doc-1', 'Doc-2', 'Doc-3'])

In [7]:
from llama_index.llms.llama_cpp import LlamaCPP
from llama_index.core import Document

# Load Mistral model with optimized generic parameters
llm = LlamaCPP(
    model_path="/content/mistral.gguf",
    temperature=0.0,  # Zero temperature for deterministic classification
    max_new_tokens=30,  # We only need a single category name
    context_window=4096,  # Increased context to handle our sampling approach
)


llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from /content/mistral.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.2
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.

In [8]:
def prepare_document_for_classification(text):
    # Instead of truncating to first 500 chars, create a better representation

    # Get first, middle, and last portions
    doc_length = len(text)
    first_part = text[:min(500, doc_length)]

    middle_start = max(0, doc_length//2 - 250)
    middle_part = text[middle_start:middle_start + min(500, doc_length - middle_start)]

    last_start = max(0, doc_length - 500)
    last_part = text[last_start:]

    # Extract any structural elements (headings, tables, etc.)
    # This is a simplified version - could use regex for better extraction
    #possible_headers = [line.strip() for line in text.split('\n')
    #                   if line.strip() and len(line.strip()) < 50
    #                   and line.strip().isupper()]
    #headers = possible_headers[:10]  # Take first 10 potential headers

    return {
        "first_part": first_part,
        "middle_part": middle_part,
        "last_part": last_part,
        "total_length": doc_length,
        #"potential_headers": "\n".join(headers)
    }

In [9]:
def classify_document(text):
    doc_info = prepare_document_for_classification(text)

    prompt = f"""You are a document classification expert. Classify this document into one of these categories:
    - Bank Statement
    - Pay Slip
    - Appraisal Report
    - Unknown

    Here's information extracted from the document:

    DOCUMENT START EXCERPT:
    {doc_info['first_part']}
    DOCUMENT START EXCERPT END

    DOCUMENT MIDDLE EXCERPT:
    {doc_info['middle_part']}
    DOCUMENT MIDDLE EXCERPT END

    DOCUMENT END EXCERPT:
    {doc_info['last_part']}
    DOCUMENT END EXCERPT END

    Total document length: {doc_info['total_length']} characters

    IMPORTANT INSTRUCTION: Your response must be EXACTLY ONE of these four options:
    Bank Statement
    Pay Slip
    Appraisal Report
    Unknown

    Do not include any explanation, reasoning, or additional text. Respond with ONLY the category name.
    """

    response = llm.complete(prompt)
    raw_response = response.text.strip()

    # Post-process to extract just the category name
    categories = ["Bank Statement", "Pay Slip", "Appraisal Report", "Unknown"]

    # First check if the response exactly matches one of our categories
    if raw_response in categories:
        return raw_response

    # If not, look for the category within the response
    for category in categories:
        if category.lower() in raw_response.lower():
            return category

    # If still no match, return the closest match
    import re
    words = re.findall(r'\b\w+\b', raw_response.lower())
    if "bank" in words or "statement" in words:
        return "Bank Statement"
    elif "pay" in words or "slip" in words or "salary" in words:
        return "Pay Slip"
    elif "appraisal" in words or "property" in words:
        return "Appraisal Report"
    else:
        return "Unknown"

In [10]:
# Classify each document
classified_docs = {}
for doc_id, text in doc_texts.items():
    doc_type = classify_document(text)
    classified_docs[doc_id] = {"text": text, "doc_type": doc_type}
    print(f"{doc_id} classified as: {doc_type}")


llama_perf_context_print:        load time =  235611.80 ms
llama_perf_context_print: prompt eval time =  235610.71 ms /  1147 tokens (  205.41 ms per token,     4.87 tokens per second)
llama_perf_context_print:        eval time =   11364.31 ms /    17 runs   (  668.49 ms per token,     1.50 tokens per second)
llama_perf_context_print:       total time =  246984.48 ms /  1164 tokens
Llama.generate: 67 prefix-match hit, remaining 879 prompt tokens to eval


Doc-1 classified as: Bank Statement


llama_perf_context_print:        load time =  235611.80 ms
llama_perf_context_print: prompt eval time =  169258.18 ms /   879 tokens (  192.56 ms per token,     5.19 tokens per second)
llama_perf_context_print:        eval time =   11674.19 ms /    18 runs   (  648.57 ms per token,     1.54 tokens per second)
llama_perf_context_print:       total time =  180941.78 ms /   897 tokens
Llama.generate: 68 prefix-match hit, remaining 686 prompt tokens to eval


Doc-2 classified as: Pay Slip


llama_perf_context_print:        load time =  235611.80 ms
llama_perf_context_print: prompt eval time =  131750.75 ms /   686 tokens (  192.06 ms per token,     5.21 tokens per second)
llama_perf_context_print:        eval time =   11966.59 ms /    19 runs   (  629.82 ms per token,     1.59 tokens per second)
llama_perf_context_print:       total time =  143727.10 ms /   705 tokens


Doc-3 classified as: Appraisal Report


In [11]:
from llama_index.core import VectorStoreIndex
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

# Load embedding model
embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
classified_docs.keys()

dict_keys(['Doc-1', 'Doc-2', 'Doc-3'])

In [13]:

# Create separate indexes for each document type
index_map = {}

for doc_id, data in classified_docs.items():
    doc_type = data["doc_type"]

    if doc_type == "Unknown":
        continue  # Skip unknown documents

    document = Document(text=data["text"], metadata={"doc_type": doc_type})

    if doc_type not in index_map:
        index_map[doc_type] = VectorStoreIndex.from_documents([document], embed_model=embed_model)
    else:
        index_map[doc_type].insert(document)

    print(f"Indexed {doc_id} as {doc_type}.")


Indexed Doc-1 as Bank Statement.
Indexed Doc-2 as Pay Slip.
Indexed Doc-3 as Appraisal Report.


In [14]:
index_map

{'Bank Statement': <llama_index.core.indices.vector_store.base.VectorStoreIndex at 0x7f7a46397450>,
 'Pay Slip': <llama_index.core.indices.vector_store.base.VectorStoreIndex at 0x7f7d08b0aa90>,
 'Appraisal Report': <llama_index.core.indices.vector_store.base.VectorStoreIndex at 0x7f7a40caddd0>}

In [16]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import CompactAndRefine
import re

def route_query(query):
    # Check which document type the query is related to
    prompt = f"""
    Classify the following question into one of these categories:
    - 'Bank Statement'
    - 'Pay Slip'
    - 'Appraisal Report'

    If it does not match any, respond with 'Unknown'.

    IMPORTANT INSTRUCTION: Your response must be EXACTLY ONE of these four options:
    Bank Statement
    Pay Slip
    Appraisal Report
    Unknown

    Do not include any explanation, reasoning, or additional text. Respond with ONLY the category name.

    Query: {query}
    """

    doc_type = llm.complete(prompt).text.strip()

    raw_response = doc_type

    # Post-process to extract just the category name
    categories = ["Bank Statement", "Pay Slip", "Appraisal Report", "Unknown"]

    # First check if the response exactly matches one of our categories
    if raw_response in categories:
        doc_type = raw_response

    # If not, look for the category within the response
    for category in categories:
        if category.lower() in raw_response.lower():
            doc_type = category

    # If still no match, return the closest match
    words = re.findall(r'\b\w+\b', raw_response.lower())
    if "bank" in words or "statement" in words:
        doc_type = "Bank Statement"
    elif "pay" in words or "slip" in words or "salary" in words:
        doc_type = "Pay Slip"
    elif "appraisal" in words or "property" in words:
        doc_type = "Appraisal Report"
    else:
        doc_type = "Unknown"

    if doc_type not in index_map:
        return "Could not determine document type."

    # Retrieve from the correct index
    retriever = index_map[doc_type].as_retriever(similarity_top_k=2)

    # Create a response synthesizer with the Mistral model
    response_synthesizer = CompactAndRefine(
        llm=llm,
        verbose=True
    )

    # Create the query engine with our explicit components
    query_engine = RetrieverQueryEngine(
        retriever=retriever,
        response_synthesizer=response_synthesizer
    )

    response = query_engine.query(query)
    return f"📄 **Document Type:** {doc_type}\n🔍 **Answer:** {response}"

# Test different queries
print(route_query("What is my net salary?"))
print(route_query("What is the appraised value of the house?"))
print(route_query("What was my last deposit?"))


Llama.generate: 1 prefix-match hit, remaining 132 prompt tokens to eval
llama_perf_context_print:        load time =  235611.80 ms
llama_perf_context_print: prompt eval time =  131829.76 ms /   644 tokens (  204.70 ms per token,     4.89 tokens per second)
llama_perf_context_print:        eval time =    4117.65 ms /     7 runs   (  588.24 ms per token,     1.70 tokens per second)
llama_perf_context_print:       total time =   28573.34 ms /   651 tokens
Llama.generate: 1 prefix-match hit, remaining 341 prompt tokens to eval
llama_perf_context_print:        load time =  235611.80 ms
llama_perf_context_print: prompt eval time =   62333.70 ms /   341 tokens (  182.80 ms per token,     5.47 tokens per second)
llama_perf_context_print:        eval time =    2922.64 ms /     5 runs   (  584.53 ms per token,     1.71 tokens per second)
llama_perf_context_print:       total time =   65259.87 ms /   346 tokens
Llama.generate: 1 prefix-match hit, remaining 137 prompt tokens to eval


📄 **Document Type:** Pay Slip
🔍 **Answer:** 9500.


llama_perf_context_print:        load time =  235611.80 ms
llama_perf_context_print: prompt eval time =   24862.35 ms /   137 tokens (  181.48 ms per token,     5.51 tokens per second)
llama_perf_context_print:        eval time =    4616.04 ms /     8 runs   (  577.00 ms per token,     1.73 tokens per second)
llama_perf_context_print:       total time =   29482.37 ms /   145 tokens
Llama.generate: 1 prefix-match hit, remaining 2532 prompt tokens to eval
llama_perf_context_print:        load time =  235611.80 ms
llama_perf_context_print: prompt eval time =  503305.44 ms /  2532 tokens (  198.78 ms per token,     5.03 tokens per second)
llama_perf_context_print:        eval time =   20598.78 ms /    29 runs   (  710.30 ms per token,     1.41 tokens per second)
llama_perf_context_print:       total time =  523918.81 ms /  2561 tokens
Llama.generate: 1 prefix-match hit, remaining 132 prompt tokens to eval


📄 **Document Type:** Appraisal Report
🔍 **Answer:** 1,650,000 (as per the document)
Query: What methods were used to determine the value of the house?


llama_perf_context_print:        load time =  235611.80 ms
llama_perf_context_print: prompt eval time =   24240.31 ms /   132 tokens (  183.64 ms per token,     5.45 tokens per second)
llama_perf_context_print:        eval time =    3503.30 ms /     6 runs   (  583.88 ms per token,     1.71 tokens per second)
llama_perf_context_print:       total time =   27747.42 ms /   138 tokens
Llama.generate: 1 prefix-match hit, remaining 1575 prompt tokens to eval
llama_perf_context_print:        load time =  235611.80 ms
llama_perf_context_print: prompt eval time =  308245.71 ms /  1575 tokens (  195.71 ms per token,     5.11 tokens per second)
llama_perf_context_print:        eval time =   16148.97 ms /    25 runs   (  645.96 ms per token,     1.55 tokens per second)
llama_perf_context_print:       total time =  324406.69 ms /  1600 tokens


📄 **Document Type:** Bank Statement
🔍 **Answer:** 2,678.39 was the last deposit made on 31st July 2018.
